# Quora Question Deduplication

Experiments on the question deduplication dataset [provided by Quora](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs). Each point in this dataset consists of two question titles and is labeled according to whether or not the titles correspond to duplicate questions.

Currently, we use a unigram-bigram bag-of-words representation for each title with a choice of PCA or an two-layer autoencoder to reduce the unwieldy dimensionality of BoW vectors. We then use Keras to define and train a deep neural network to classify pairs of titles as duplicate or non-duplicate.

### Setup

In [4]:
%matplotlib inline
import cPickle as pickle
import heapq
import matplotlib.pyplot as plt
import numpy as np
import re
import sys
import tensorflow as tf

from collections import defaultdict
from sklearn.decomposition import PCA

In [5]:
def rawDataGen(fname):
    """Generator over rows from the raw Quora dataset file. Yields dictionaries with keys
    defined by the headers in the first row."""
    with open(fname) as f:
        fields = f.readline().strip().split("\t")
        line = f.readline()
        while line != "":
            yield {field:attr for field, attr in zip(fields, line.strip().split("\t"))}
            line = f.readline()

def preprocessText(text):
    """Currently converts to lower case, converts numbers to "#", separates out punctuation,
    consolidates multiple whitespaces to one, and splits into tokens."""
    out = text.lower()
    out = re.sub(r"\d+", "#", out)
    out = re.sub(r"([(\.\.\.)\.!?:;,])", r" \1 ", out)
    out = re.sub(r"\s+", " ", out)
    return out.split()

def tokensToBow(tokens, vocab):
    """Given a unigram/bigram vocabulary and a list of tokens, generates a list
    representing a bag-of-words featurization of the tokens."""
    bow = [0 for _ in range(len(vocab))]
    for tok in tokens:
        unigram = tuple([tok])
        if unigram in vocab.keys():
            bow[vocab[unigram]] += 1
    for i in range(len(tokens) - 1):
        bigram = tuple(tokens[i:i+2])
        if bigram in vocab.keys():
            bow[vocab[bigram]] += 1
    return bow

def analyzeDataset(dataFname):
    """Preprocesses the questions in each row of the dataset and builds
    a word count based on the resulting token sequences."""
    wordCounts = defaultdict(int)
    outputDocs = list()
    numProcessed = 0
    for doc in rawDataGen(dataFname):
        try:
            q1Processed = preprocessText(doc["question1"])
            q2Processed = preprocessText(doc["question2"])
            for nGramSize in (1, 2):
                for tokens in (q1Processed, q2Processed):
                    for idx in range(len(tokens) - (nGramSize - 1)):
                        nGram = tuple(tokens[idx:idx + nGramSize])
                        wordCounts[nGram] += 1
            outputDocs.append({
                "q1": q1Processed,
                "q2": q2Processed,
                "label": doc["is_duplicate"]
            })
            numProcessed += 1
            if numProcessed % 1000 == 0:
                sys.stdout.write("\r%d rows analyzed." % (numProcessed))
                sys.stdout.flush()
        except:
            pass
    sys.stdout.write("\n")
    return outputDocs, wordCounts

def generateVocab(counts, vocabSize=2000):
    """Given a dictionary of word/ngram counts, generates a vocabulary consisting of
    the top vocabSize words."""
    sys.stdout.write("Generating vocab... ")
    sys.stdout.flush()
    countPairs = list(wordCounts.iteritems())
    countPairs = [(term, count) for term, count in countPairs if count > 5]
    topN = heapq.nlargest(vocabSize, countPairs, key=lambda pair: pair[1])
    vocab = {term:idx for idx, (term, count) in enumerate(topN)}
    sys.stdout.write("done!\n")
    return vocab

def finalizeDataset(analyzedDocs, vocab, maxRows=100000):
    """Builds a bag-of-words featurization for both Q1 and Q2 datasets as well
    as an array of labels and returns all three."""
    q1List, q2List, labelsList = list(), list(), list()
    numProcessed = 0
    for doc in analyzedDocs[:maxRows]:
        q1List.append(tokensToBow(doc["q1"], vocab))
        q2List.append(tokensToBow(doc["q2"], vocab))
        labelsList.append(int(doc["label"]))
        numProcessed += 1
        if numProcessed % 100 == 0:
            sys.stdout.write("\r%d docs processed." % (numProcessed))
            sys.stdout.flush()
    sys.stdout.write("\n")
    return np.array(q1List), np.array(q2List), np.array(labelsList)

def saveData(dataSpec, q1, q2, labels, vocab):
    """Save the Q1/Q2/labels arrays and the vocabulary to files. Objects are
    saved to `<dataSpec>_data.npz` and `<dataSpec>_vocab.pkl`."""
    with open(dataSpec + "_data.npz", "w") as dataF:
        data = np.savez(dataF,
                        q1=q1,
                        q2=q2,
                        labels=labels
                        )
    with open(dataSpec + "_vocab.pkl", "w") as vocabF:
        pickle.dump(vocab, vocabF)

def loadData(dataSpec):
    """Reads from the files saved by saveData."""
    with open(dataSpec + "_data.npz") as dataF:
        data = np.load(dataF)
        q1 = data["q1"]
        q2 = data["q2"]
        labels = data["labels"]
    with open(dataSpec + "_vocab.pkl") as vocabF:
        vocab = pickle.load(vocabF)
    return q1, q2, labels, vocab

### Data prep
We use a simple unigram-bigram bag-of-words representation for now. More complex featurizations would include using an LSTM encoder, either on one-hot vectors or on learned vector representations.

In [ ]:
analyzedDocs, wordCounts = analyzeDataset("./quora_duplicate_questions.tsv")
vocab = generateVocab(wordCounts, vocabSize=5000)
q1, q2, labels = finalizeDataset(analyzedDocs, vocab, maxRows=1000000)

In [ ]:
saveData("unigram_bigram_5k", q1, q2, labels, vocab)

In [6]:
q1, q2, labels, vocab = loadData("unigram_bigram_5k")

### Dimensionality reduction
First, we try PCA to 1000 dimensions.

In [ ]:
allData = np.concatenate((q1[:10000, :], q2[:10000, :]))
pca = PCA(n_components=1000)
pca.fit(allData)

In [ ]:
with open("pca.pkl", "w") as pcaF:
    pickle.dump(pca, pcaF)

In [7]:
with open("pca.pkl") as pcaF:
    pca = pickle.load(pcaF)

In [8]:
# Reduce to to 1000 dimensions using PCA.
nSmall = 200000
q1Small = pca.transform(q1[:nSmall,:])
q2Small = pca.transform(q2[:nSmall,:])
labelsSmall = labels[:nSmall]

We also experiment briefly with autoencoders. This is a simple two-layer fully-connected autoencoder to 1000 dimensions.

In [9]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization

autoencData = np.concatenate((q1[:100000], q2[:100000]))

autoenc = Sequential([
        Dense(2000, input_shape=(autoencData.shape[1],)),
        BatchNormalization(),
        Dense(1000),
        BatchNormalization(),
        Dense(2000),
        Dense(5000)
    ])

autoenc.compile(loss="mse", optimizer="adam")
autoenc.fit(autoencData, autoencData, validation_split=0.1, nb_epoch=1, batch_size=250)

Using TensorFlow backend.


Train on 180000 samples, validate on 20000 samples
Epoch 1/1
180000/180000 [==============================] - 556s - loss: 0.0042 - val_loss: 0.0013


In [12]:
# Example from the autoencoder
print np.array([autoencData[100:101], autoenc.predict(autoencData[100:101])]).T

[[[ 2.          1.99352646]]

 [[ 1.          0.99872744]]

 [[ 1.          0.94655061]]

 ..., 
 [[ 0.          0.01668848]]

 [[ 0.         -0.03442896]]

 [[ 0.         -0.01198013]]]


In [ ]:
# Building the encoder using the weights from the trained autoencoder
enc = Sequential([
        Dense(2000, input_shape=(5000,)),
        BatchNormalization(),
        Dense(1000),
    ])
enc.set_weights(autoenc.get_weights()[:8])
q1Small = enc.predict(q1)
q2Small = enc.predict(q2)

### Classification
Now, we try the duplicate/non-duplicate classification task using the dimensionality-reduced data. We use a 4-layer fully-connected neural network with ReLU activation on the hidden layers and a softmax readout layer, as well as dropout and batch normalization layers to improve generalization and training time, respectively.

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam

clfDataSmall = np.concatenate((q1Small, q2Small), axis=1)

model = Sequential([
        Dense(2000, activation="relu", input_shape=(clfDataSmall.shape[1],)),
        Dropout(0.5),
        Dense(1000, activation="relu"),
        Dense(200, activation="relu"),
        BatchNormalization(),
        Dense(2, activation="softmax")
    ])

labelsSmallOneHot = np.array([1 - labelsSmall, labelsSmall]).T

model.compile(loss="categorical_crossentropy", optimizer=Adam(1e-2), metrics=["accuracy", "fbeta_score"])
model.fit(clfDataSmall, labelsSmallOneHot, validation_split=0.1, nb_epoch=3, batch_size=250)

Train on 180000 samples, validate on 20000 samples
Epoch 1/3
180000/180000 [==============================] - 129s - loss: 0.5349 - acc: 0.7301 - fbeta_score: 0.7301 - val_loss: 0.4923 - val_acc: 0.7505 - val_fbeta_score: 0.7505
Epoch 2/3
180000/180000 [==============================] - 130s - loss: 0.4609 - acc: 0.7759 - fbeta_score: 0.7759 - val_loss: 0.4624 - val_acc: 0.7725 - val_fbeta_score: 0.7724
Epoch 3/3
180000/180000 [==============================] - 133s - loss: 0.4134 - acc: 0.8020 - fbeta_score: 0.8020 - val_loss: 0.4602 - val_acc: 0.7765 - val_fbeta_score: 0.7765


For reference, we look at the distribution of labels in the dataset. We see below that 63% of pairs are non-duplicates and 37% are duplicates.

In [18]:
print labelsSmallOneHot.sum(0).astype(float) / labelsSmallOneHot.shape[0]

[ 0.62758  0.37242]
